# Create an Iceberg Table using Amazon Athena and AWS Glue Catalog
This will create an Athena table in the Glue Catalog (Hive Metastore) which allows us to query the dataset files in S3. We will create a table in the Glue Catalog based on the `Amazon Customer Reviews Dataset` in S3. We will be using the Apache Iceberg format to create an ACID compiant data source in S3 which will support various capabilities like in-line updates and time travel.


# Upload Maven Jar's to S3

In [5]:
!curl -O https://repo1.maven.org/maven2/software/amazon/awssdk/bundle/2.15.40/bundle-2.15.40.jar
!curl -O https://repo1.maven.org/maven2/org/apache/iceberg/iceberg-spark-runtime-3.1_2.12/0.13.1/iceberg-spark-runtime-3.1_2.12-0.13.1.jar
!curl -O https://repo1.maven.org/maven2/software/amazon/awssdk/url-connection-client/2.15.40/url-connection-client-2.15.40.jar

Welcome to the Glue Interactive Sessions Kernel
For more information on available magic commands, please type %help in any new cell.

Please view our Getting Started page to access the most up-to-date information on the Interactive Sessions kernel: https://docs.aws.amazon.com/glue/latest/dg/interactive-sessions.html
It looks like there is a newer version of the kernel available. The latest version is 0.37.2 and you have 0.37.0 installed.
Please run `pip install --upgrade aws-glue-sessions` to upgrade your kernel
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  245M  100  245M    0     0   248M      0 --:--:-- --:--:-- --:--:--  247M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 21.0M  100 21.0M    0     0   126M      0 --:--:-- --:--:-- --:--:--  127M
  % Tota

In [9]:
!aws s3 cp iceberg-spark-runtime-3.1_2.12-0.13.1.jar s3://sagemaker-us-east-1-371366150581/glue-iceberg-jars/iceberg-spark-runtime-3.1_2.12-0.13.1.jar
!aws s3 cp bundle-2.15.40.jar s3://sagemaker-us-east-1-371366150581/glue-iceberg-jars/bundle-2.15.40.jar
!aws s3 cp url-connection-client-2.15.40.jar s3://sagemaker-us-east-1-371366150581/glue-iceberg-jars/url-connection-client-2.15.40.jar

upload: ./iceberg-spark-runtime-3.1_2.12-0.13.1.jar to s3://sagemaker-us-east-1-371366150581/glue-iceberg-jars/iceberg-spark-runtime-3.1_2.12-0.13.1.jar
upload: ./bundle-2.15.40.jar to s3://sagemaker-us-east-1-371366150581/glue-iceberg-jars/bundle-2.15.40.jar
upload: ./url-connection-client-2.15.40.jar to s3://sagemaker-us-east-1-371366150581/glue-iceberg-jars/url-connection-client-2.15.40.jar


In [11]:
!aws s3 ls s3://sagemaker-us-east-1-371366150581/glue-iceberg-jars/

2023-03-16 17:24:12  257939967 bundle-2.15.40.jar
2023-03-16 17:24:10   22123750 iceberg-spark-runtime-3.1_2.12-0.13.1.jar
2023-03-16 17:24:15      21027 url-connection-client-2.15.40.jar


# Setup the Glue Interactive Session

In [13]:
%stop_session

There is no current session.


In [19]:
%session_id_prefix iceberg-to-sagemaker
%additional_python_modules seaborn,psutil,sagemaker
%number_of_workers 10
%extra_jars s3://sagemaker-us-east-1-371366150581/glue-iceberg-jars/bundle-2.15.40.jar,s3://sagemaker-us-east-1-371366150581/glue-iceberg-jars/iceberg-spark-runtime-3.1_2.12-0.13.1.jar,s3://sagemaker-us-east-1-371366150581/glue-iceberg-jars/url-connection-client-2.15.40.jar
%glue_version 3.0

Setting session ID prefix to iceberg-to-sagemaker
Additional python modules to be included:
seaborn
psutil
sagemaker
Previous number of workers: 5
Setting new number of workers to: 10
Extra jars to be included:
s3://sagemaker-us-east-1-371366150581/glue-iceberg-jars/bundle-2.15.40.jar
s3://sagemaker-us-east-1-371366150581/glue-iceberg-jars/iceberg-spark-runtime-3.1_2.12-0.13.1.jar
s3://sagemaker-us-east-1-371366150581/glue-iceberg-jars/url-connection-client-2.15.40.jar
Setting Glue version to: 3.0
s3://sagemaker-us-east-1-371366150581/glue-iceberg-jars/bundle-2.15.40.jar,s3://sagemaker-us-east-1-371366150581/glue-iceberg-jars/iceberg-spark-runtime-3.1_2.12-0.13.1.jar,s3://sagemaker-us-east-1-371366150581/glue-iceberg-jars/url-connection-client-2.15.40.jar


In [21]:
%%configure
{
    "conf": "spark.sql.catalog.glue_catalog.catalog-impl=org.apache.iceberg.aws.glue.GlueCatalog --conf spark.sql.catalog.glue_catalog.warehouse=s3://sagemaker-us-east-1-371366150581/iceberg/ --conf spark.sql.extensions=org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions --conf spark.sql.catalog.glue_catalog=org.apache.iceberg.spark.SparkCatalog --conf spark.sql.catalog.glue_catalog.io-impl=org.apache.iceberg.aws.s3.S3FileIO --conf spark.sql.catalog.glue_catalog.lock-impl=org.apache.iceberg.aws.glue.DynamoLockManager --conf spark.sql.catalog.glue_catalog.lock.table=demo-iceberg-gis",
}

The following configurations have been updated: {'conf': 'spark.sql.catalog.glue_catalog.catalog-impl=org.apache.iceberg.aws.glue.GlueCatalog --conf spark.sql.catalog.glue_catalog.warehouse=s3://sagemaker-us-east-1-371366150581/iceberg/ --conf spark.sql.extensions=org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions --conf spark.sql.catalog.glue_catalog=org.apache.iceberg.spark.SparkCatalog --conf spark.sql.catalog.glue_catalog.io-impl=org.apache.iceberg.aws.s3.S3FileIO --conf spark.sql.catalog.glue_catalog.lock-impl=org.apache.iceberg.aws.glue.DynamoLockManager --conf spark.sql.catalog.glue_catalog.lock.table=demo-iceberg-gis'}


In [1]:
spark

Authenticating with environment variables and user-defined glue_role_arn: arn:aws:iam::371366150581:role/SageMakerRepoRole
Trying to create a Glue session for the kernel.
Worker Type: G.1X
Number of Workers: 10
Session ID: iceberg-to-sagemaker-95b6b78c-c30e-4c4d-a040-6cc0ecaf8770
Job Type: glueetl
Applying the following default arguments:
--glue_kernel_version 0.37.0
--enable-glue-datacatalog true
--additional-python-modules seaborn,psutil,sagemaker
--extra-jars s3://sagemaker-us-east-1-371366150581/glue-iceberg-jars/bundle-2.15.40.jar,s3://sagemaker-us-east-1-371366150581/glue-iceberg-jars/iceberg-spark-runtime-3.1_2.12-0.13.1.jar,s3://sagemaker-us-east-1-371366150581/glue-iceberg-jars/url-connection-client-2.15.40.jar
--conf spark.sql.catalog.glue_catalog.catalog-impl=org.apache.iceberg.aws.glue.GlueCatalog --conf spark.sql.catalog.glue_catalog.warehouse=s3://sagemaker-us-east-1-371366150581/iceberg/ --conf spark.sql.extensions=org.apache.iceberg.spark.extensions.IcebergSparkSessionE

# Create the Iceberg Database in Glue Catalog

In [2]:
catalog = "glue_catalog"
database = "iceberg_reviews"
spark.sql(f"CREATE DATABASE IF NOT EXISTS {catalog}.{database}")

DataFrame[]


In [3]:
dbs = spark.sql(f"SHOW DATABASES")
assert database in dbs.toPandas()['namespace'].values, "Database has not been created propertly"

# Write Data to Iceberg Format
**This section will take a few minutes as we are transfering the data source to your S3 bucket**

In [4]:
df_parquet = spark.read.parquet("s3://amazon-reviews-pds/parquet/")

In [5]:
df_parquet.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: string (nullable = true)
 |-- product_title: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: date (nullable = true)
 |-- year: integer (nullable = true)
 |-- product_category: string (nullable = true)


In [6]:
table = 'reviews'
df_parquet.writeTo(f"{catalog}.{database}.{table}").tableProperty("format-version", "2").createOrReplace()

# Verify The Table Has Been Created Succesfully

In [7]:
statement = f"SHOW TABLES in {catalog}.{database}"
df_tables = spark.sql(statement).toPandas()
df_tables

         namespace tableName
0  iceberg_reviews   reviews


# Run A Sample Query

In [8]:
query = f"""
SELECT product_category, product_title, star_rating from {catalog}.{database}.{table}
WHERE product_category = "Shoes"
"""
df = spark.sql(query)
df.show(10)

+----------------+--------------------+-----------+
|product_category|       product_title|star_rating|
+----------------+--------------------+-----------+
|           Shoes|Reebok Men's Oakl...|          5|
|           Shoes|Dr. Scholl's Wome...|          1|
|           Shoes|Kenneth Cole REAC...|          4|
|           Shoes|Rockport Cobb Hil...|          4|
|           Shoes|Yak Pak Megu Hand...|          5|
|           Shoes|Crocs Kids' Handl...|          5|
|           Shoes|Teva Women's Mush...|          5|
|           Shoes|Liebeskind Berlin...|          5|
|           Shoes|Rachel Shoes Litt...|          5|
|           Shoes|Clarks Men's Skyw...|          4|
+----------------+--------------------+-----------+
only showing top 10 rows


In [9]:
empty = True if df.count() == 0 else False
if empty:
    print("++++++++++++++++++++++++++++++++++++++++++++++++++++++")
    print("[ERROR] YOUR DATA HAS NOT BEEN REGISTERED WITH ATHENA.")
    print("LOOK IN PREVIOUS CELLS TO FIND THE ISSUE.             ")
    print("++++++++++++++++++++++++++++++++++++++++++++++++++++++")
else:
    print("[OK]")

[OK]
